In [2]:
import tensorflow as tf 

import tensorflow_datasets as tfds 
DATA_DIR = "./tensorflow-datasets"


In [3]:
%load_ext autoreload
%autoreload 2

In [46]:
import model
import util
import pandas as pd 
from pathlib import Path


vocab_ds = util.get_for_vocab_ds()

results_l = pd.read_csv("results_lstm_/exp_results.csv")
results_g = pd.read_csv("results_gru_/exp_results.csv")
vectorizer = util.get_text_vectorizer(vocab_ds, vocab_size=30000, sequence_length=1200)

combined = pd.concat([results_l,results_g], ignore_index=True, sort=False)
with_regularizer = combined[combined['regularizer']==True]
best= with_regularizer[with_regularizer['accuracy']==with_regularizer['accuracy'].max()]
best_config = best['model'].iloc[0]
if "gru" in best_config:
    best_model = model.get_bidirectional_gru_attention(vectorizer)
else:
    best_model = model.get_bidirectional_lstm_attention(vectorizer)
best_model.load_weights(best_config)
test_ds= util.get_test_ds()
accuracy = util.evaluate_model(best_model, test_ds,Path("./"))
print("Best model")
print(best)
print("Test accuracy", accuracy)


None


2024-11-10 20:12:24.307069: W tensorflow/core/util/tensor_slice_reader.cc:95] Could not open results_lstm_/32_0.001_True/best_model.hd5: Failed precondition: results_lstm_/32_0.001_True/best_model.hd5; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


Best model
   Unnamed: 0  batch_size  learning_rate  regularizer  accuracy  \
2           2          32          0.001         True    0.8889   

                                                logs  \
2  {'loss': [1.1264721155166626, 0.57569921016693...   

                                        model  
2  results_lstm_/32_0.001_True/best_model.hd5  
Test accuracy 0.87552
